# 기본 그래프 생성

이번 튜토리얼에서는 LangGraph를 사용하여 그래프를 생성하는 방법을 배웁니다.

LangGraph 의 그래프를 정의하기 위해서는

1. State 정의
2. 노드 정의
3. 그래프 정의
4. 그래프 컴파일
5. 그래프 시각화

단계를 거칩니다.

그래프 생성시 조건부 엣지를 사용하는 방법과 다양한 흐름 변경 방법을 알아봅니다.

![langgraph-building-graphs](assets/langgraph-building-graphs.png)

## State 정의

In [1]:
from typing import TypedDict, Annotated, List
from langchain_core.documents import Document
import operator


# State 정의
class GraphState(TypedDict):
    context: Annotated[List[Document], operator.add]
    answer: Annotated[List[Document], operator.add]
    question: Annotated[str, "user question"]
    binary_score: Annotated[str, "binary score yes or no"]

## 노드 정의

In [2]:
def retrieve(state: GraphState) -> GraphState:
    # retrieve: 검색
    # retriever.invoke(question["question"])
    documents = "검색된 문서 (retrieve)"
    return {"context": documents}


def rewrite_query(state: GraphState) -> GraphState:
    # Query Transform: 쿼리 재작성
    documents = "검색된 문서 (rewrite_query)"
    return GraphState(context=documents)


def call_gpt(state: GraphState) -> GraphState:
    # LLM 실행
    answer = "GPT 생성된 답변"
    return GraphState(answer=answer)


def call_claude(state: GraphState) -> GraphState:
    # LLM 실행
    answer = "Claude 의 생성된 답변"
    return GraphState(answer=answer)


def relevance_check(state: GraphState) -> GraphState:
    # Relevance Check: 관련성 확인
    binary_score = "Relevance Score"
    return GraphState(binary_score=binary_score)


def sum_up(state: GraphState) -> GraphState:
    # sum_up: 결과 종합
    answer = "종합된 답변"
    return GraphState(answer=answer)


def search_on_web(state: GraphState) -> GraphState:
    # Search on Web: 웹 검색
    documents = state["context"] = "기존 문서"
    searched_documents = "검색된 문서"
    documents += searched_documents
    return GraphState(context=documents)


def decision(state: GraphState) -> GraphState:
    # 의사결정
    decision = "결정"
    # 로직을 추가할 수 가 있고요.

    if state["binary_score"] == "yes":
        return "종료"
    else:
        return "재검색"

In [4]:
## 그래프 정의

In [24]:
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from share.langgraph.graphs import visualize_graph

# (1): Conventional RAG
# (2): 재검색
# (3): 멀티 LLM
# (4): 쿼리 재작성


# langgraph.graph에서 StateGraph와 END를 가져옵니다.
workflow = StateGraph(GraphState)

# 노드를 추가합니다.
workflow.add_node("retrieve", retrieve)

# workflow.add_node("rewrite_query", rewrite_query)  # (4)

workflow.add_node("call_gpt", call_gpt)
# workflow.add_node("call_claude", call_claude)  # (3)
workflow.add_node("gpt_relevance_check", relevance_check)
# workflow.add_node("claude_relevance_check", relevance_check)  # (3)
workflow.add_node("결과 종합", sum_up)

# 각 노드들을 연결합니다.
workflow.add_edge("retrieve", "call_gpt")
# workflow.add_edge("retrieve", "call_claude")  # (3)
# workflow.add_edge("rewrite_query", "retrieve")  # (4)
workflow.add_edge("call_gpt", "gpt_relevance_check")
workflow.add_edge("gpt_relevance_check", "결과 종합")
# workflow.add_edge("call_claude", "claude_relevance_check")  # (3)
# workflow.add_edge("claude_relevance_check", "결과 종합")  # (3)

# workflow.add_edge("결과 종합", END)  # (2) - off

# 조건부 엣지를 추가합니다. (2), (4)
# workflow.add_conditional_edges(
#     "결과 종합",  # 관련성 체크 노드에서 나온 결과를 is_relevant 함수에 전달합니다.
#     decision,
#     {
#         "재검색": "retrieve",  # 관련성이 있으면 종료합니다.
#         "종료": END,  # 관련성 체크 결과가 모호하다면 다시 답변을 생성합니다.
#     },
# )

# 조건부 엣지를 추가합니다. (4)
# workflow.add_conditional_edges(
#     "결과 종합",  # 관련성 체크 노드에서 나온 결과를 is_relevant 함수에 전달합니다.
#     decision,
#     {
#         "재검색": "rewrite_query",  # 관련성이 있으면 종료합니다.
#         "종료": END,  # 관련성 체크 결과가 모호하다면 다시 답변을 생성합니다.
#     },
# )

# 시작점을 설정합니다.
workflow.set_entry_point("retrieve")

# 기록을 위한 메모리 저장소를 설정합니다.
memory = MemorySaver()

# 그래프를 컴파일합니다.
app = workflow.compile(checkpointer=memory)

In [25]:
# 그래프를 시각화 합니다.

# import nest_asyncio
# from share.langgraph.graphs import visualize_graph_local

# 그래프 시각화
visualize_graph(app)

# nest_asyncio.apply()

# visualize_graph_local(app)
# generate_mermaid_png(app.get_graph(), "out.png")

[ERROR] Visualize Graph Error: name '_DEFAULT_BLOCKSIZE' is not defined


그래프를 시각화 합니다.